In [1]:
from autosearch.agents.graphrag_search import GraphragSearch
import asyncio
import json
from typing import List, Dict
import nest_asyncio
nest_asyncio.apply()



In [2]:
questions = [
"""
Create a comprehensive table comparing representation learning models for polymers. Include the following information for each model:

Model name
Year introduced
Key authors/research group
Type of representation (e.g., graph-based, sequence-based, 3D structure-based)
Input features used
Dimensionality of the learned representation
Training data type and size
Key applications (e.g., property prediction, polymer design)
Notable advantages
Limitations or challenges

After the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.
""",
"""
Create a comparative analysis of machine learning models used for predicting glass transition temperature (Tg) and melting temperature (Tm) of polymers. Include the following in your analysis:

A table comparing at least 5 different models, with the following information for each:
a) Model name and type (e.g., Random Forest, Neural Network, etc.)
b) Year introduced
c) Key authors/research group
d) Input features or representation used
e) Dataset size and composition
f) Accuracy metrics for Tg prediction (e.g., R², RMSE, MAE)
g) Accuracy metrics for Tm prediction (e.g., R², RMSE, MAE)
h) Computational complexity or training time (if available)
A brief discussion (2-3 paragraphs) on:
a) The overall trends in prediction accuracy for Tg vs Tm
b) Factors that contribute to higher accuracy in certain models
c) Challenges in predicting these properties accurately
d) Recent advancements or novel approaches in the field
A short section on potential areas for improvement and future research directions.

Please provide citations for the models and studies mentioned in your analysis.
"""
]

models = ['gpt-4', 'gpt-4o', 'gpt-4o-mini', 'gpt-35-turbo', 'gpt-35-turbo-16k']

# input_dir = '/home/azureuser/autogen_uscases/graphrag_memo/output/20240830-145930/artifacts'
input_dir = '/home/azureuser/autogen_uscases/autosearch/notebooks/graphrag/output/20240903-050706/artifacts'
# input_dir = '/home/azureuser/autogen_uscases/graphrag_memo/output/20240830-145930/artifacts'
lancedb_uri = "/home/azureuser/autogen_uscases/autosearch/notebooks/graphrag/output/20240903-050706/artifacts/lancedb"
embedding_model = 'text-embedding-3-small'



In [3]:
import json
from typing import List
import concurrent.futures
from datetime import datetime
import asyncio

def run_searches(searcher, q):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    local_result, global_result = loop.run_until_complete(asyncio.gather(
        searcher.local_search(query=q),
        searcher.global_search(query=q)
    ))
    loop.close()
    return local_result, global_result

def process_model(m, questions, input_dir, lancedb_uri, embedding_model):
    print(f"Model: {m}")
    config = {
        "GRAPHRAG_LLM_MODEL": m,
        "GRAPHRAG_LLM_DEPLOYMENT_NAME": m,
        "GRAPHRAG_EMBEDDING_DEPLOYMENT_NAME": embedding_model,
        "GRAPHRAG_EMBEDDING_MODEL": embedding_model,
    }
    searcher = GraphragSearch(
        input_dir=input_dir,
        lancedb_uri=lancedb_uri,
        config=config
    )
    
    model_results = []
    for q in questions:
        print(f"Question: {q}")
        local_result, global_result = run_searches(searcher, q)
        model_results.append({
            "question": q,
            "model": m,
            "local_result": local_result.response,
            "global_result": global_result.response,
            "global_llm_tokens": global_result.prompt_token,
            "global_llm_calls": global_result.llm_calls

        })
        print(f"Completed: Question: {q}, Model: {m}")
    
    return model_results

def eval_models(questions: List[str], models: List[str], input_dir: str, lancedb_uri: str, embedding_model: str, number_workers: int):
    results = []
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=number_workers) as executor:
        future_to_model = {executor.submit(process_model, m, questions, input_dir, lancedb_uri, embedding_model): m for m in models}
        for future in concurrent.futures.as_completed(future_to_model):
            model = future_to_model[future]
            try:
                model_results = future.result()
                results.extend(model_results)
            except Exception as exc:
                print(f'{model} generated an exception: {exc}')

    # Generate a unique filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'results_{timestamp}.json'

    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"Results saved to {filename}")

    return results

In [4]:

results = eval_models(questions, models, input_dir, lancedb_uri, embedding_model, number_workers=1)


Model: gpt-4


/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Question: 
Create a comprehensive table comparing representation learning models for polymers. Include the following information for each model:

Model name
Year introduced
Key authors/research group
Type of representation (e.g., graph-based, sequence-based, 3D structure-based)
Input features used
Dimensionality of the learned representation
Training data type and size
Key applications (e.g., property prediction, polymer design)
Notable advantages
Limitations or challenges

After the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.



Completed: Question: 
Create a comprehensive table comparing representation learning models for polymers. Include the following information for each model:

Model name
Year introduced
Key authors/research group
Type of representation (e.g., graph-based, sequence-based, 3D structure-based)
Input features used
Dimensionality of the learned representation
Training data type and size
Key applications (e.g., property prediction, polymer design)
Notable advantages
Limitations or challenges

After the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.
, Model: gpt-4
Question: 
Create a comparative analysis of machine learning models used for predicting glass transition temperature (Tg) and melting temperature (Tm) of polymers. Include the following in your analysis:

A table comparing at least 5 diffe

Completed: Question: 
Create a comparative analysis of machine learning models used for predicting glass transition temperature (Tg) and melting temperature (Tm) of polymers. Include the following in your analysis:

A table comparing at least 5 different models, with the following information for each:
a) Model name and type (e.g., Random Forest, Neural Network, etc.)
b) Year introduced
c) Key authors/research group
d) Input features or representation used
e) Dataset size and composition
f) Accuracy metrics for Tg prediction (e.g., R², RMSE, MAE)
g) Accuracy metrics for Tm prediction (e.g., R², RMSE, MAE)
h) Computational complexity or training time (if available)
A brief discussion (2-3 paragraphs) on:
a) The overall trends in prediction accuracy for Tg vs Tm
b) Factors that contribute to higher accuracy in certain models
c) Challenges in predicting these properties accurately
d) Recent advancements or novel approaches in the field
A short section on potential areas for improvement an

...

Question: 
Create a comprehensive table comparing representation learning models for polymers. Include the following information for each model:

Model name
Year introduced
Key authors/research group
Type of representation (e.g., graph-based, sequence-based, 3D structure-based)
Input features used
Dimensionality of the learned representation
Training data type and size
Key applications (e.g., property prediction, polymer design)
Notable advantages
Limitations or challenges

After the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.



Exception in _map_response_single_batch
Traceback (most recent call last):
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/structured_search/global_search/search.py", line 214, in _map_response_single_batch
    search_response = await self.llm.agenerate(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/llm/oai/chat_openai.py", line 142, in agenerate
    async for attempt in retryer:
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/asyncio/__init__.py", line 166, in __anext__
    do = await self.iter(retry_state=self._retry_state)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/asyncio/__init__.py", line 153, in iter
    result = await action(retry_state)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/_utils.py", line 99, in inner
 

Completed: Question: 
Create a comprehensive table comparing representation learning models for polymers. Include the following information for each model:

Model name
Year introduced
Key authors/research group
Type of representation (e.g., graph-based, sequence-based, 3D structure-based)
Input features used
Dimensionality of the learned representation
Training data type and size
Key applications (e.g., property prediction, polymer design)
Notable advantages
Limitations or challenges

After the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.
, Model: gpt-35-turbo-16k
Question: 
Create a comparative analysis of machine learning models used for predicting glass transition temperature (Tg) and melting temperature (Tm) of polymers. Include the following in your analysis:

A table comparing at le

Exception in _map_response_single_batch
Traceback (most recent call last):
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/structured_search/global_search/search.py", line 214, in _map_response_single_batch
    search_response = await self.llm.agenerate(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/graphrag/query/llm/oai/chat_openai.py", line 142, in agenerate
    async for attempt in retryer:
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/asyncio/__init__.py", line 166, in __anext__
    do = await self.iter(retry_state=self._retry_state)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/asyncio/__init__.py", line 153, in iter
    result = await action(retry_state)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/anaconda/envs/autoget/lib/python3.12/site-packages/tenacity/_utils.py", line 99, in inner
 

Completed: Question: 
Create a comparative analysis of machine learning models used for predicting glass transition temperature (Tg) and melting temperature (Tm) of polymers. Include the following in your analysis:

A table comparing at least 5 different models, with the following information for each:
a) Model name and type (e.g., Random Forest, Neural Network, etc.)
b) Year introduced
c) Key authors/research group
d) Input features or representation used
e) Dataset size and composition
f) Accuracy metrics for Tg prediction (e.g., R², RMSE, MAE)
g) Accuracy metrics for Tm prediction (e.g., R², RMSE, MAE)
h) Computational complexity or training time (if available)
A brief discussion (2-3 paragraphs) on:
a) The overall trends in prediction accuracy for Tg vs Tm
b) Factors that contribute to higher accuracy in certain models
c) Challenges in predicting these properties accurately
d) Recent advancements or novel approaches in the field
A short section on potential areas for improvement an

In [5]:
results

[{'question': '\nCreate a comprehensive table comparing representation learning models for polymers. Include the following information for each model:\n\nModel name\nYear introduced\nKey authors/research group\nType of representation (e.g., graph-based, sequence-based, 3D structure-based)\nInput features used\nDimensionality of the learned representation\nTraining data type and size\nKey applications (e.g., property prediction, polymer design)\nNotable advantages\nLimitations or challenges\n\nAfter the table, provide a brief analysis (2-3 paragraphs) of the current trends in polymer representation learning, highlighting the most promising approaches and areas for future research. Please cite relevant papers for each model and in your analysis.\n',
  'model': 'gpt-4',
  'local_result': 'Unfortunately, I do not have access to external databases or the ability to pull in specific publication details such as authors, years, or paper titles. However, I can provide a table based on the infor